Вывод текста везде (кроме tika) закомменчен для удобства скроллинга

<p>В данном ноутбуке рассмотрены несколько библиотек для извлечения текста из pdf, doc, docx и rtf файлов</p>
<p>Рассмотренные библиотеки: Apache Tika, PyPDF2, PyMuPDF (fitz), pdfminer.six, python-docx, docx2txt и способ чтения без дополнительных библиотек; так же рассмотрен способ конвертации файлов docx->pdf с помощью библиотеки docx2pdf </p>

In [37]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.36 s (started: 2022-12-28 19:52:39 +03:00)


# Apache Tika

Для работы обязательно нужны установленная Java SE Development Kit 19.0.1 (по требованиям версия 1.6 и выше) и jar-файл (лежит в директории src);
ссылка на <a href="https://www.oracle.com/java/technologies/javase/jdk19-archive-downloads.html">java</a><href > 

<p>Перед запуском функции get_document_text необходимо запустить с помощью java jar-файл C:\Users\username\AppData\Local\Temp\tika-server.jar </p>
<p>Команда для запуска в командной строке java -jar C:\Users\username\AppData\Local\Temp\tika-server.jar </p>
<p>После выполнения все должно работать</p>
<p>Принимаемые функцией get_document_text расширения файлов: .doc, .docx, .pdf, .rtf; Но вообще tika способна работать с практически любыми расширениями файлов (медиа, текст и т.д.) </p>
<p></p>
<p>После окончания работы с tika необходимо выключить открытый jar-файл (для этого вроде как достаточно закрыть cmd)</p>

In [1]:
!pip install tika

In [22]:
import tika
from tikapp import TikaApp
tika.initVM()

In [33]:
from tika import parser

def get_document_text(filepath):
    doc_type = filepath.split('.')[-1]
    if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
        text = parser.from_file(filepath)['content']
    else:
        print(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
    return text.strip()

time: 0 ns (started: 2022-12-28 19:49:08 +03:00)


In [38]:
path_doc="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"
print(get_document_text(path_doc))

2022-12-28 20:06:18,724 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-28 20:06:18,725 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.


RuntimeError: Unable to start Tika server.

time: 1.16 s (started: 2022-12-28 20:06:17 +03:00)


In [35]:
path_docx="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx"
print(get_document_text(path_docx))

Образец простого договора аренды квартиры, заключаемого между физическим лицами

ДОГОВОР НАЙМА КВАРТИРЫ
	г. _______________
	«____» ______________ 2022 г.





 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ___________________________________

In [ ]:
path_pdf="C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
print(get_document_text(path_pdf))

## **Библиотека PyPDF2.**

In [3]:
!pip install pypdf2
from PyPDF2 import PdfFileReader

time: 10.1 s (started: 2022-12-28 18:17:02 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 10.8 с, но при этом шрифт с тенью выводится в единственном экземпляре (+)

In [4]:
pdf_pypdf2 = 'C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'

with open(pdf_pypdf2, "rb") as filehandle:  
    pdf = PdfFileReader(filehandle)
   
    info = pdf.getDocumentInfo()
    pages = pdf.getNumPages()
#     print("Количество страниц в документе: %i\n\n" % pages)
#     print("Мета-описание: ", info)

    for i in range(pages):
        page = pdf.getPage(i)
#         print("Стр.", i+1, " мета: ", page, "\n\nСодержание;\n")
#         print(page.extractText())

time: 328 ms (started: 2022-12-28 18:17:12 +03:00)


## **PyMuPDF (он же fitz)**

In [5]:
!pip install pymupdf
import fitz

time: 10.3 s (started: 2022-12-28 18:17:12 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 53 мс

In [6]:
pdf_fitz = "C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
text = ''
with fitz.open(pdf_fitz) as doc:
    for page in doc:
        text += page.get_text()
text = '\n'.join(text.split('\n')[::2])
# print(text)

time: 1.53 s (started: 2022-12-28 18:17:22 +03:00)


В fitz можно раскрашивать конкретные слова

In [46]:
text = ''
highlight_text = ['Договор','Квартир']
doc = fitz.open(pdf_fitz)
for current_page in doc:
    for word in highlight_text:
        text_instances = current_page.search_for(word)
        for inst in text_instances:
            highlight = current_page.add_highlight_annot(inst)
            highlight.set_colors({"stroke":(0, 0.5, 0)})
            highlight.update()
doc.save("C:/Users/265375/Desktop/Work/parsing/Договор_аренды_автомобильного_прицепа_colored.pdf", garbage=4, deflate=True, clean=True)

time: 31 ms (started: 2022-12-27 20:57:00 +03:00)


## **pdfminer.six (версия pdfminer для python 3.*)**

In [7]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text

time: 9.95 s (started: 2022-12-28 18:17:24 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 2.18 с

In [8]:
pdf_pdfminer = 'C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'
text_pdfminer = extract_text(pdf_pdfminer)
text_pdfminer = '\n'.join(text_pdfminer.split('\n')[::2])
# print(text_pdfminer)

The PDF <_io.BufferedReader name='C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


time: 1.06 s (started: 2022-12-28 18:17:34 +03:00)


## **Чтение docx файлов**

## **python-docx (docx)**

In [9]:
!pip install python-docx
import docx

time: 13 s (started: 2022-12-28 18:17:35 +03:00)


Не смог разобраться, почему не работает

In [54]:
doc = 'C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
getText(doc)

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

time: 203 ms (started: 2022-12-27 21:00:08 +03:00)


## **docx2txt**

In [10]:
!pip install docx2txt 
import docx2txt

time: 8.74 s (started: 2022-12-28 18:17:48 +03:00)


Сохраняет форматирование, время чтения и вывода - 18 мс

In [11]:
text = docx2txt.process("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx")
# print(text)

time: 234 ms (started: 2022-12-28 18:17:57 +03:00)


## **Простой метод, использующий стандартную библиотеку zipfile**

In [12]:
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

time: 141 ms (started: 2022-12-28 18:17:57 +03:00)


Основан на python-docx, не требует дополнительной установки библиотек, простая функция для чтения, время чтения и вывода - 11.7 мс

In [13]:
WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


def get_docx_text(path):
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                 for node in paragraph.iter(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)

time: 125 ms (started: 2022-12-28 18:17:57 +03:00)


In [14]:
doc = 'C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
# print(get_docx_text(doc))

time: 235 ms (started: 2022-12-28 18:17:57 +03:00)


In [15]:
!pip install aspose-words
import aspose.words as aw


time: 17.9 s (started: 2022-12-28 18:17:57 +03:00)


In [16]:
path1="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"

doc = aw.Document(path1)
saveOptions = aw.saving.PdfSaveOptions()
saveOptions.compliance = aw.saving.PdfCompliance.PDF17 
doc.save("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf", saveOptions)
pdf_fitz = "C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf"

doc = fitz.open(pdf_fitz)
# print("Исходный документ: ", doc)
# print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
# print(doc.metadata)
text = ''
for current_page in range(doc.page_count):
    text += doc.get_page_text(current_page,"text")
text = '\n'.join(text.split('\n')[::2])
# print(text)

time: 51.3 s (started: 2022-12-28 18:18:15 +03:00)


# Конвертер docx -> pdf

In [17]:
!pip install docx2pdf
from docx2pdf import convert

time: 10.6 s (started: 2022-12-28 18:19:07 +03:00)


In [65]:
convert("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx", "C:/Users/265375/Desktop/Work/parsing/test/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]

time: 891 ms (started: 2022-12-27 21:04:32 +03:00)
